In [3]:
!pip install pyspark
!pip install -U -q PyDrive
!apt update
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 987.4/987.4 kB 16.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:2 https://cli.github.com/packages stable InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,065 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubun

In [4]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

# Creamos el Spark Context

In [6]:
# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [18]:
DATA_PATH = "/content/drive/MyDrive/CienciaDeDatos/TP1/data/"

sqlContext = SQLContext(sc)

rdd_order_items = (
    sqlContext.read.csv(DATA_PATH + "order_items.csv", header=True, inferSchema=True)
    .select("order_id", 'line_total', "quantity")
    .rdd
    )


/usr/local/lib/python3.12/dist-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


# 7) Obtener el total de órdenes, volumen total de productos vendidos, promedio de productos por orden, y extremos tanto en cantidad de ítems como en precios individuales.

In [19]:
# ------------------- Promedio, máximos y mínimos -------------------
by_order = (
    rdd_order_items
    .filter(lambda r: r["quantity"] and r["line_total"])
    .map(lambda r: (int(r["order_id"]), int(r["quantity"])))
    .reduceByKey(lambda a, b: a + b)
)

# Métricas de cantidad
total_items = by_order.map(lambda x: x[1]).sum()
total_orders = by_order.count()
avg_items = total_items / (total_orders if total_orders else 0)

max_items = by_order.map(lambda x: x[1]).max()
min_items = by_order.map(lambda x: x[1]).min()

max_price = rdd_order_items.filter(lambda r: r["line_total"]).map(lambda x: float(x["line_total"])).max()
min_price = rdd_order_items.filter(lambda r: r["line_total"]).map(lambda x: float(x["line_total"])).min()

# ------------------- Resultados -------------------
print("--------------------- Resumen de órdenes ----------------------")
print(f"Total de órdenes: {total_orders}")
print(f"Total de ítems vendidos: {int(total_items)}")
print(f"Promedio de ítems por orden: {avg_items:.2f}")
print(f"Mínimo de ítems en una orden: {min_items}")
print(f"Máximo de ítems en una orden: {max_items}")
print(f"Precio mínimo registrado: ${min_price:.2f}")
print(f"Precio máximo registrado: ${max_price:.2f}")
print("---------------------------------------------------------------")


--------------------- Resumen de órdenes ----------------------
Total de órdenes: 90883
Total de ítems vendidos: 1139620
Promedio de ítems por orden: 12.54
Mínimo de ítems en una orden: 1
Máximo de ítems en una orden: 75
Precio mínimo registrado: $1.14
Precio máximo registrado: $29932.30
---------------------------------------------------------------


# Conclusiones:

Los resultados muestran que, en promedio, cada orden incluye unas 12,5 unidades, aunque existe una gran dispersión entre pedidos: algunas órdenes contienen un solo ítem, mientras que otras llegan a 75. Esto sugiere la coexistencia de distintos perfiles de compra, probablemente clientes minoristas y otros de tipo mayorista o corporativo.

El rango de precios también es muy amplio, desde apenas $1,14 hasta más de $29.000, lo que refleja una oferta de productos muy heterogénea en valor.

Como hipótesis, se puede suponer que la distribución de cantidades por orden está sesgada hacia valores bajos, con unas pocas órdenes de gran tamaño que elevan el promedio general, los famosos outliers. Además, los precios extremos podrían corresponder a categorías de producto muy distintas, o a la presencia de artículos premium dentro del catálogo.

